In [1]:
import os

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from models.xswin import XNetSwinTransformer
from models.modules import SwinResidualCrossAttention

from models.modules.normal.residual_cross_attention import _extract_windows, _unfold_padding_prep, _fold_unpadding_prep

from torchinfo import summary


In [2]:
*A, B= 1, 2
A


[1]

In [3]:
x = torch.tensor([
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
])

x = torch.tensor([
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
])

# print(torch.cat((x, x), dim=-1))


# x = torch.vstack((torch.hstack((x, x*2)), torch.hstack((x*3, x*4))))
x = torch.stack((x, x*2, x*3, x*4))
x = torch.vstack((torch.hstack((x, x*2)), torch.hstack((x*3, x*4))))


# x = x.reshape(2, 2, -1)
x = x.unsqueeze(0)

print(x.shape)

x = _extract_windows(x, 4, 4)

# print(x.shape)


torch.Size([1, 8, 8, 5])


In [4]:
H_DIM = 16

r1 = torch.ones((5, 10, 22, H_DIM))
r2 = torch.ones(r1.shape)

f1, pinfo = _unfold_padding_prep(r1, 4, 4)
print(f1.shape)
f2 = _fold_unpadding_prep(f1, pinfo)
print(f2.shape)

SRCA = SwinResidualCrossAttention(
    [4, 4],
    H_DIM,
    4,
)

SRCA(r1, r2).shape


torch.Size([5, 12, 24, 16])
torch.Size([5, 10, 22, 16])


torch.Size([5, 10, 22, 16])

In [5]:
patch_size = [4, 4]
embed_dim = 64
depths = [1, 1, 1]
num_heads = [4, 8, 16]
window_size = [4, 4]
num_classes = 1

IMG_H, IMG_W = 151, 309

global_stages = 1
input_size = [IMG_H, IMG_W]
final_downsample = True
residual_cross_attention = True

swin = XNetSwinTransformer(patch_size, embed_dim, depths, 
                           num_heads, window_size, num_classes=num_classes,
                           global_stages=global_stages, input_size=input_size,
                           final_downsample=final_downsample, residual_cross_attention=residual_cross_attention,
                           )


x = torch.randn((5, 3, IMG_H, IMG_W))
print(x.shape)

# print(swin)

y = swin(x)
print(y.shape)

summary(swin, input_size=[1, 3, IMG_H, IMG_W])
# 512 25
# Parameter containing:
# tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
#          [0., 0., 0.,  ..., 0., 0., 0.],
#          [0., 0., 0.,  ..., 0., 0., 0.],
#          ...,
#          [0., 0., 0.,  ..., 0., 0., 0.],
#          [0., 0., 0.,  ..., 0., 0., 0.],
#          [0., 0., 0.,  ..., 0., 0., 0.]]])
# (625, 512)

# 20 39
# 10 19
# 5 9


None
torch.Size([5, 3, 151, 309])


TypeError: unsupported operand type(s) for +: 'Tensor' and 'NoneType'

In [ ]:
print(swin)


XNetSwinTransformer(
  (smooth_conv_in): ConvolutionTriplet(
    (layers): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.01)
    )
  )
  (patching): Patching(
    (patching): Sequential(
      (0): Conv2d(64, 64, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
  )
  (encoder): ModuleList(
    (0): Sequential(
      (0): SwinTrans